# Homework 7 (30 marks)



Import all the libraries you require in the cell below.


In [52]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# Enter your code here
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    classification_report,
    recall_score,
    precision_score,
    accuracy_score
)

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

## Implement the Ensemble methods learnt in class and compare their accuarcies.

The dataset you are going to be using for homework is the **Wisconsin Breast Cancer dataset (cancer.csv)**

The dataset contains a total number of 10 features labeled in either benign or malignant classes. The features have 699 instances out of which 16 feature values are missing. The dataset only contains numeric values.

Attribute Information:

1. Sample code number: id number
2. Clump Thickness: 1 - 10
3. Uniformity of Cell Size: 1 - 10
4. Uniformity of Cell Shape: 1 - 10
5. Marginal Adhesion: 1 - 10
6. Single Epithelial Cell Size: 1 - 10
7. Bare Nuclei: 1 - 10
8. Bland Chromatin: 1 - 10
9. Normal Nucleoli: 1 - 10
10. Mitoses: 1 - 10
11. Class: (2 for benign, 4 for malignant) (**target variable**)

For more information: https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)

### 1. Read the dataset into variable called '**data**' (1 mark)

In [53]:
pd.set_option('display.max_columns', 100)
# Enter your code here

data = pd.read_csv("cancer.csv")

### **Preprocessing**: Data needs to be preprocessed before implementing ensemble methods. It is done for you here. 
### Run the below code first and then answer the questions from 2 - 7.

#### Deleting unnecessary columns: The column "Sample code number" is just an indicator and it's of no use in the modeling. So, let's drop it:


In [54]:
data.drop(['Sample Code Number'],axis = 1, inplace = True)


#### Handling missing values : 
As mentioned earlier, the dataset contains missing values. The column named "Bare Nuclei" contains them. The missing values are represneted as "?". 

Replace those "?"s with 0's and impute them with Mean Imputation

In [55]:
data['Bare Nuclei']

0       3
1       3
2       3
3       3
4       3
       ..
694     1
695     1
696     8
697    10
698    10
Name: Bare Nuclei, Length: 699, dtype: int64

In [56]:
data.replace('?',0, inplace=True)

In [57]:
# Convert the DataFrame object into NumPy array otherwise you will not be able to impute
values = data.values
# Now impute it
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputedData = imputer.fit_transform(values)

#### Normalizing the data:
Ranges of the features of the dataset are not the same. This may cause a problem. A small change in a feature might not affect the other. To address this problem, normalize the ranges of the features to a uniform range, in this case, 0 - 1.

In [58]:
scaler = MinMaxScaler(feature_range=(0, 1))
normalizedData = scaler.fit_transform(imputedData)
cols = ['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bland Chromatin', 'Bare Nuclei', 'Normal Nucleoli', 'Mitosis','Class']
normalizedData = pd.DataFrame(normalizedData, columns=cols)
print(normalizedData.head())

   Clump Thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
0         0.444444                 0.000000                  0.000000   
1         0.444444                 0.333333                  0.333333   
2         0.222222                 0.000000                  0.000000   
3         0.555556                 0.777778                  0.777778   
4         0.333333                 0.000000                  0.000000   

   Marginal Adhesion  Single Epithelial Cell Size  Bland Chromatin  \
0           0.000000                     0.111111              0.1   
1           0.444444                     0.666667              1.0   
2           0.000000                     0.111111              0.2   
3           0.000000                     0.222222              0.4   
4           0.222222                     0.111111              0.1   

   Bare Nuclei  Normal Nucleoli  Mitosis  Class  
0     0.222222         0.000000      0.0    0.0  
1     0.222222         0.111111      0.0

### Data preprocessing is done and now you will answer the below questions using the **normalizedData**: 

### 2. Split the data into test and training data with test size - 30%. Compute the baseline classification accuracy for X_train. (3 marks)

In [59]:
# Enter your code here
X = normalizedData.iloc[:,:-1]
y = normalizedData['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y)

dummy_classifier = DummyClassifier(strategy='most_frequent')
dummy_classifier.fit(X_train,y_train)
baseline_acc = dummy_classifier.score(X_test,y_test)

print("Baseline Accuracy = ", baseline_acc)

Baseline Accuracy =  0.6571428571428571


### 3.  Bagging : Build a generic Bagging ensemble and print the accuracy (4 marks)
---


Hyperparameters:

Base estimator = DecisionTreeClassifier

n_estimators = 10

random_state = 42

---


In [60]:
# Generic Bagging model
# Enter your code here

#bagging
model_bagging = BaggingClassifier(base_estimator = DecisionTreeClassifier(), n_estimators = 10, random_state = 42)
model_bagging.fit(X_train, y_train)
pred_bagging = model_bagging.predict(X_test)
acc_bagging = accuracy_score(y_test, pred_bagging)

print('Accuracy = ', acc_bagging)
print('\nClassification Report:\n')
print(classification_report(y_test, pred_bagging))

Accuracy =  0.9523809523809523

Classification Report:

              precision    recall  f1-score   support

         0.0       0.96      0.97      0.96       138
         1.0       0.94      0.92      0.93        72

    accuracy                           0.95       210
   macro avg       0.95      0.94      0.95       210
weighted avg       0.95      0.95      0.95       210



### 4. RandomForest : (7 marks)
#### a) Build a Random Forest model and print the accuracy (4 marks)
---

Constructor arguments: 


n_estimators = 100, max_features = 7 and random_state = 42 


---




In [61]:
# Random Forest model
# Enter your code here
model_rf = RandomForestClassifier(n_estimators=100, max_features=7, random_state=42)
model_rf.fit(X_train, y_train)
predict_rf = model_rf.predict(X_test)

acc_forest = accuracy_score(y_test, predict_rf)

print('Accuracy = ', acc_forest)
print('\nClassification Report:\n')
print(classification_report(y_test, predict_rf))

Accuracy =  0.9476190476190476

Classification Report:

              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96       138
         1.0       0.94      0.90      0.92        72

    accuracy                           0.95       210
   macro avg       0.95      0.94      0.94       210
weighted avg       0.95      0.95      0.95       210



####  b) Calculate the top 3 important features for the above **RandomForest** model and print them (3 marks)

In [62]:
# Top 3 features for RandomForest
# Enter your code here

feature_importances = model_rf.feature_importances_
features = X_train.columns
df = pd.DataFrame({'features': features, 'importance': feature_importances}).nlargest(3, 'importance')
print(df)

                   features  importance
1   Uniformity of Cell Size    0.479737
2  Uniformity of Cell Shape    0.220095
5           Bland Chromatin    0.148706


### 5. Boosting: (7 marks)
#### a) Build an AdaBoost model with training data and print the accuracy (4 marks)
---

Hyperparameters:

Base estimator = DecisionTreeClassifier, max_depth = 4

n_estimators = 200

random_state = 42

learning_rate = 0.05


---









In [63]:
# AdaBoost Classification
# Enter your code here
base_est = DecisionTreeClassifier(max_depth =4)

ada_boost = AdaBoostClassifier(base_est, n_estimators=200, random_state=42, learning_rate=.05)
ada_boost.fit(X_train, y_train)

acc_boost = accuracy_score(y_test, ada_boost.predict(X_test))
print('Accuracy = ', acc_boost)

 Accuracy =  0.9476190476190476


#### b) Calculate the top 3 important features for the above **AdaBoost** model and print them (3 marks)

In [64]:
# Top 3 features for AdaBoost
# Enter your code here

feature_importances = ada_boost.feature_importances_
features = X_train.columns
df = pd.DataFrame({'features': features, 'importance': feature_importances}).nlargest(3, 'importance')
print(df)

                      features  importance
2     Uniformity of Cell Shape    0.220802
0              Clump Thickness    0.169351
4  Single Epithelial Cell Size    0.150803


### 6. Voting : Using a voting classifier, build an ensemble of RandomForestClassifier, DecisionTreeClassifier, Support Vector Machine and Logistic Regression. (7 marks)


---


Use max_depth = 4, n_estimators = 200, voting = soft

In [65]:
# Voting Ensemble for Classification
# Enter your code here
rfClf = RandomForestClassifier(n_estimators=200, random_state=0) 
dtClf = DecisionTreeClassifier(max_depth =4)
svmClf = SVC(probability=True, random_state=0)
logClf = LogisticRegression(random_state=0)

clf2 = VotingClassifier(estimators = [('rf',rfClf), ('dt',dtClf), ('svm',svmClf), ('log', logClf)], voting='soft') 

clf2.fit(X_train, y_train)

clf2_pred = clf2.predict(X_test)

print('Accuracy = ', accuracy_score(y_test, clf2_pred))

Accuracy =  0.9571428571428572


### 7. Mention the best model among the above 4 models and its accuracy (1 mark)

In [81]:
# the best model is the model with the has the highest accuracy score 

print('Bagging Accuracy = ', acc_bagging)
print('Random Forest Accuracy = ', acc_forest)
print('AdaBoost Accuracy = ', acc_boost)
print('Voting Accuracy = ', accuracy_score(y_test, clf2_pred))

bestModel = {'Bagging': acc_bagging, 'Random Forest': acc_forest, 'AdaBoost': acc_boost, 'Voting': accuracy_score(y_test, clf2_pred)}
print("\nBest Model Accuracy:", max(bestModel, key = bestModel.get))

Bagging Accuracy =  0.9523809523809523
Random Forest Accuracy =  0.9476190476190476
AdaBoost Accuracy =  0.9476190476190476
Voting Accuracy =  0.9571428571428572

Best Model Accuracy: Voting
